# Computation of cutting planes

# The set-up

In [2]:
import numpy as np
import accpm
%load_ext autoreload
%autoreload 1
%aimport accpm

$\DeclareMathOperator{\domain}{dom}
\newcommand{\transpose}{\text{T}}
\newcommand{\vec}[1]{\begin{pmatrix}#1\end{pmatrix}}$

# Example 1

To test the computation of cutting planes we consider the problem
\begin{align*}
    &\text{minimize} \quad f_\text{obj}(x_0, x_1) = (x_0 - 5)^2 + (x_1 - 5)^2 \\
    &\phantom{\text{minimize}} \quad f_0(x_0, x_1) = 
    a_0^\transpose x - b_0 = \vec{1\\0}^\transpose \vec{x_0\\x_1} - 20 = x_0 - 20 \leq 0\\
    &\phantom{\text{minimize}} \quad f_1(x_0, x_1) =
    a_1^\transpose x - b_1 = \vec{-1\\0}^\transpose \vec{x_0\\x_1} = -x_0 \leq 0\\
    &\phantom{\text{minimize}} \quad f_2(x_0, x_1) =
    a_2^\transpose x - b_2 = \vec{0\\1}^\transpose \vec{x_0\\x_1} - 20 = x_1 - 20 \leq 0 \\
    &\phantom{\text{minimize}} \quad f_3(x_0, x_1) = 
    a_3^\transpose x - b_3 = \vec{0\\-1}^\transpose \vec{x_0\\x_1} = -x_1 \leq 0,
\end{align*}
which clearly has the solution $x^\star = (x_1^\star, x_2^\star) = (5, 5)$. Now, the gradients of the objective function and constraint functions are 
\begin{align*}
    &\nabla f_\text{obj}(x_0, x_1) = \vec{2(x_0 - 5)\\2(x_1 - 5)}, \\
    &\nabla f_0(x_0, x_1) = \vec{1\\0}, \quad \nabla f_1(x_0, x_1) = \vec{-1\\0}, \\
    &\nabla f_2(x_0, x_1) = \vec{0\\1}, \quad \nabla f_3(x_0, x_1) = \vec{0\\-1}.
\end{align*}
We implement these functions as follows:

In [3]:
def funcobj(x):
    return (x[0]-5)**2 + (x[1]-5)**2
    
def func0(x):
    return x[0] - 20

def func1(x):
    return -x[0]
    
def func2(x):
    return x[1] - 20
    
def func3(x):
    return -x[1]

def grad_funcobj(x):
    return np.array([2*(x[0] - 5), 2*(x[1] - 5)])

def grad_func0(x):
    return np.array([1, 0])

def grad_func1(x):
    return np.array([-1, 0])

def grad_func2(x):
    return np.array([0, 1])
    
def grad_func3(x):
    return np.array([0, -1])

A = np.array([[1, 0],[-1,0],[0,1],[0,-1]])
b = np.array([20, 0, 20, 0])

The ACCPM requires that the initial polygon $\mathcal{P}_0$ (here I've abused terminology and by the initial polygon $\mathcal{P}_0$ I actually mean the system of linear inequalities $Ax \leq b$) contain the $\epsilon$-suboptimal set for some given $\epsilon > 0$. It is clear that if we take
\begin{align*}
    A = \vec{a_0^\transpose\\a_1^\transpose\\a_2^\transpose\\a_3^\transpose}, b = \vec{20\\0\\20\\0}
\end{align*}
this will be satisfied for some reasonably small enough $\epsilon > 0$, as we know the solution is $x^\star = (x_0^\star, x_1^\star) = (5, 5) \in [0, 20] \times [0, 20]$. 

Now, we start with $k=0$.  
Now, $x^{(0)}_{ac}$ is the solution of the minimization problem 
\begin{equation*}
    \min_{\domain \phi} \phi(x) = - \sum_{i=0}^{3}{\log{(b_i - a_i^\transpose x)}}.
\end{equation*}
So, we solve the problem
\begin{align*}
    &\phantom{iff}\nabla \phi(x) = \sum_{i=0}^{3
    } \frac{1}{b_i - a_i^\transpose x}a_i = 0 \\
    &\iff \frac{1}{20-x_0}\begin{bmatrix}1\\0\end{bmatrix} + \frac{1}{x_0}\begin{bmatrix}-1\\0\end{bmatrix} + \frac{1}{20-x_1}\begin{bmatrix}0\\1\end{bmatrix} + \frac{1}{x_1}\begin{bmatrix}0\\-1\end{bmatrix} = 0 \\
    &\iff \frac{1}{20-x_0} - \frac{1}{x_0} = 0, \frac{1}{20-x_1} - \frac{1}{x_1} = 0 \\
    &\iff x_0 = \frac{20}{2} = 10, x_1 = \frac{20}{2} = 10,
\end{align*}
and conclude $x^{(0)}_{ac} = (10, 10)$. We then query the oracle at $x^{(0)}_{ac}$. (Here, $f_\text{best} = f_\text{obj}(10, 10) = 50$ since this is the $0$-th iteration.) We know the point is feasible and therefore
\begin{align*}
    &a_4 = \nabla f_\text{obj}(10, 10) = \vec{10\\10}, \\
    &b_4 = \nabla f_\text{obj}(10, 10)^\transpose \vec{10\\10} = \vec{10\\10}^\transpose \vec{10\\10} = 200,
\end{align*}
which we normalize to get
\begin{align*}
    &a_4 =  \frac{1}{\sqrt{100^2 + 100^2}} \nabla f_\text{obj}(10, 10) 
         = \vec{\frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} } \approx \vec{0.7071 \\ 0.7071}, \\
    &b_4 = \frac{1}{\sqrt{100^2 + 100^2}} \nabla f_\text{obj}(10, 10)^\transpose \vec{10\\10} = \vec{10\\10}^\transpose \vec{10\\10} = \frac{20}{\sqrt{2}} = 10\sqrt{2} \approx 14.1421,
\end{align*}
and therefore update
\begin{align*}
    A = \vec{a_0^\transpose\\a_1^\transpose\\a_2^\transpose\\a_3^\transpose\\a_4^\transpose}, b = \vec{20\\0\\20\\0\\10\sqrt{2}}, k = 1.
\end{align*}
Now, $x^{(1)}_{ac}$ is the solution of the minimization problem 
\begin{equation*}
    \min_{\domain \phi} \phi(x) = - \sum_{i=0}^{4}{\log{(b_i - a_i^\transpose x)}}.
\end{equation*}
So, we solve the problem
\begin{align*}
    &\phantom{iff}\nabla \phi(x) = \sum_{i=0}^{4
    } \frac{1}{b_i - a_i^\transpose x}a_i = 0 \\
    &\iff \frac{1}{20-x_0}\vec{1\\0} + \frac{1}{x_0}\vec{-1\\0} + \frac{1}{20-x_1}\vec{0\\1} + \frac{1}{x_1}\vec{0\\-1} + \frac{\sqrt{2}}{20-x_0-x_1} \vec{\frac{1}{\sqrt{2}}\\\frac{1}{\sqrt{2}}} = 0\\
    &\iff \frac{1}{20-x_0} - \frac{1}{x_0} + \frac{1}{20 - x_0- x_1}= 0, \frac{1}{20-x_1} - \frac{1}{x_1} + \frac{1}{20 - x_0- x_1} = 0 \\
    &\iff x_0 = x_1 = 2(5 \pm \sqrt{5}) \approx 14.4721 \text{ or } 5.52786,
\end{align*}
and take $x^{(1)}_{ac} = (2(5-\sqrt{5}), 2(5-\sqrt{5})) \approx (5.52786, 5.52786)$. We then query the
oracle at $x^{(1)}_{ac}$. Here 
$f_\text{obj}(x^{(1)}_{ac}) = f_\text{obj}(2(5-\sqrt{5}), 2(5-\sqrt{5})) = 90 - 40\sqrt{5} \approx 0.557281 \leq f_\text{best} = 50$ so we update 
$f_\text{best} = 90 - 40\sqrt{5} \approx 0.557281$ and therefore put (and normalize)  
\begin{align*}
    &a_5 = \frac{1}{\sqrt{2(10-4\sqrt{5})^2}} \nabla f_\text{obj}(x^{(1)}_{ac}) = 
           \frac{1}{\sqrt{2(10-4\sqrt{5})^2}} \nabla f_\text{obj}\vec{2(5-\sqrt{5}) \\ 2(5-\sqrt{5})} =
           \frac{1}{\sqrt{2(10-4\sqrt{5})^2}} \vec{10-4\sqrt{5}\\10-4\sqrt{5}}
           = \vec{\frac{1}{\sqrt{2}}\\\frac{1}{\sqrt{2}}}\approx \vec{0.7071 \\ 0.7071}, \\
    &b_5 = \frac{1}{\sqrt{2(10-4\sqrt{5})^2}} \nabla f_\text{obj}(x^{(1)}_{ac})^\transpose \vec{2(5-\sqrt{5}) \\ 2(5-\sqrt{5})} = 
           \frac{1}{\sqrt{2}} \vec{1\\1}^\transpose \vec{2(5-\sqrt{5}) \\ 2(5-\sqrt{5})} = 2\sqrt{2} (5 - \sqrt{5})
           \approx 7.8176,
\end{align*}
updating $A$ and $b$ also. Iteration $k = 1$ is concluded by incrementing $k$. We see that this computation will only get more complicated and test our function by simply computing the analytic center and evaluating the gradient at it.  

In [32]:
 accpm.accpm(funcobj, (func0, func1, func2, func3), A, b, 
             grad_func=grad_funcobj, 
             grad_constr=(grad_func0, grad_func1, grad_func2, grad_func3),
             alpha=0.01, beta=0.7, start=1, maxiter = 20, testing=True)

Initially: b = [ 20.   0.  20.   0.] and A =
 [[ 1.  0.]
 [-1.  0.]
 [ 0.  1.]
 [ 0. -1.]]
----------------
At iteration 0 AC computation SUCCEEDED with AC [10. 10.] where
        a_cp = [ 0.7071  0.7071] and b_cp = [ 14.1421]
At iteration 1 AC computation SUCCEEDED with AC [ 5.5279  5.5278] where
        a_cp = [ 0.7071  0.7071] and b_cp = [ 7.8176]
At iteration 2 AC computation SUCCEEDED with AC [ 3.0259  3.0262] where
        a_cp = [-0.7072 -0.7071] and b_cp = [-5.5755]
At iteration 3 AC computation SUCCEEDED with AC [ 4.7661  4.7653] where
        a_cp = [-0.7058 -0.7084] and b_cp = [-6.7397]
At iteration 4 AC computation SUCCEEDED with AC [ 5.1532  5.2582] where
        a_cp = [ 0.5104  0.8599] and b_cp = [ 7.1519]
At iteration 5 AC computation SUCCEEDED with AC [ 7.8834  2.3826] where
        a_cp = [ 0.7404 -0.6721] and b_cp = [ 2.3003]
At iteration 6 AC computation SUCCEEDED with AC [ 5.2255  4.7611] where
        a_cp = [ 0.6865 -0.7271] and b_cp = [ 0.0984]
At iteration 7 AC

We know the desired solution is $x^\star = (5, 5)$ and see that at iteration 8 we have $x^{(8)}_\text{ac} = (4.9291, 5.0319)$ which is very good and at iteration 12 we have $x^{(12)}_\text{ac} = (5.0022, 5.009)$ which is even better.